In [ ]:
# === ONE-CELL: SOUL-INK (Conversational, empathetic coach — LITE, no HF) ===
%pip -q install streamlit SpeechRecognition pydub plotly >/dev/null
!apt-get -yqq install ffmpeg >/dev/null

APP = r"""
import os, io, time, tempfile, random
from datetime import datetime
import streamlit as st
import plotly.express as px

# ---------------- UI ----------------
st.set_page_config(page_title="SOUL-INK — Conversational Coach", page_icon="💜", layout="wide")
st.markdown(r'''
<style>
:root { --ink:#a855f7; }
.stApp { background: radial-gradient(1200px 800px at 20%% 10%%, #1b0f28 0%%, #0b0712 60%%, #08060d 100%%); }
h1,h2,h3,h4 { color:#e9d5ff!important; }
.stButton>button { background: var(--ink); color:white; border-radius:12px; }
.stChatMessage { background: rgba(168,85,247,.08); border:1px solid rgba(168,85,247,.25); padding:10px 14px; border-radius:12px; }
</style>
''', unsafe_allow_html=True)

st.title("💜 SOUL-INK — The AI that really *listens*")
st.caption("Anonymous by design • voice or text • one gentle question at a time")
st.info("Not therapy or medical advice. If you feel unsafe, contact local support.")

# ---------------- State ----------------
if "msgs" not in st.session_state:
    st.session_state.msgs = [{"role":"coach","text":"Hey—good to see you. How are you *really* doing right now?"}]
if "entries" not in st.session_state: st.session_state.entries=[]
if "phase" not in st.session_state: st.session_state.phase="opener"   # opener -> explore1 -> explore2 -> offer -> skill -> wrap
if "last_emotion" not in st.session_state: st.session_state.last_emotion="neutral"
if "topic" not in st.session_state: st.session_state.topic=None

# ---------------- Tiny heuristic "emotion" ----------------
VOCAB = {
  "joy":["happy","grateful","love","excited","good","relief","proud","cheerful","optimistic"],
  "sadness":["sad","down","lonely","cry","loss","tired","worthless","regret","overwhelmed","empty","lost"],
  "anger":["angry","mad","annoyed","rage","hate","furious","pissed","irritated","resent","frustrated"],
  "fear":["scared","afraid","anxious","nervous","worry","panic","terrified","pressure","stress","stressed"],
  "shame":["ashamed","guilty","embarrass","disgusted with myself","failure","bad person","not enough"],
}
def detect_emotion(t:str):
    t=t.lower()
    scores={k:0 for k in VOCAB}
    for k,ws in VOCAB.items():
        for w in ws:
            if w in t: scores[k]+=1
    if sum(scores.values())==0: return "neutral"
    return max(scores, key=scores.get)

# ---------------- Response engine (natural tone) ----------------
EMP_PREFIX = {
  "joy":   ["You sound upbeat.","I’m hearing some lightness.","Seems like something went well."],
  "sadness":["That sounds heavy.","I can hear the weight in that.","That hurts—thanks for saying it out loud."],
  "anger": ["That sounds frustrating.","I get why that would piss you off.","Your boundary feels crossed there."],
  "fear":  ["That sounds tense.","Anxiety can be loud.","Uncertainty is uncomfortable—I’m here."],
  "shame": ["That self-talk sounds harsh.","You’re being tough on yourself.","That’s a painful story to carry."],
  "neutral":["Got it.","I’m with you.","I’m listening."],
}
ASK_1 = {
  "joy":   ["What happened there? Tell me the part that mattered.","What sparked that feeling today?"],
  "sadness":["What happened right before this feeling showed up?","If this feeling had a headline, what would it say?"],
  "anger": ["What exactly crossed the line?","If you could name the core ‘unfair’ thing, what is it?"],
  "fear":  ["What’s the worry saying word-for-word?","What feels at risk here?"],
  "shame": ["Whose standards are you measuring yourself against?","If a friend was in your shoes, would you judge them the same?"],
  "neutral":["What’s the main thing on your mind right now?","Where do you want to start?"],
}
ASK_2 = {
  "joy":   ["What would repeating 10%% of that look like tomorrow?","Who helped make that possible?"],
  "sadness":["What do you wish you had in this moment?","Where do you feel this most in your body?"],
  "anger": ["What would a fair ask sound like in one sentence?","Under the anger—do you notice hurt or fear?"],
  "fear":  ["How likely is the feared outcome (0–100%%)?","What would ‘prepared’ look like for the next hour?"],
  "shame": ["What evidence pushes back against that harsh story?","What would a kinder version of this thought say?"],
  "neutral":["What small outcome would make today less messy?","If we solved one slice, which slice matters most?"],
}
OFFER = [
  "Do you want a quick calm-down trick, or should we keep unpacking for a bit?",
  "Want a 60-second grounding exercise, or another follow-up question first?",
  "Should we try a tiny action you can do in five minutes, or keep talking?"
]
CALM_TRICKS = {
  "60s Grounding": "Look around: 5 things you can see • 4 you can touch • 3 you can hear • 2 you can smell • 1 you can taste. Breathe slowly as you name them.",
  "Box Breathing": "Inhale 4 • hold 4 • exhale 4 • hold 4. Repeat for 1–2 minutes.",
  "Name & Reframe": "Write a single sentence: ‘I’m feeling ___ because ___; right now I can ___ for 5 minutes.’"
}
TINY_STEPS = {
  "Send a boundary text": "“I feel __ when __. I need __. If not, I’ll __.” Draft it—no sending needed.",
  "Five-minute task": "Pick one tiny task that would make the next hour 1%% better. Start it. Stop at 5 minutes.",
  "Self-compassion note": "Write to yourself the way you’d write to a close friend about this exact situation."
}

def empathetic_reply(user_text:str):
    emo = detect_emotion(user_text)
    st.session_state.last_emotion = emo
    prefix = random.choice(EMP_PREFIX.get(emo, EMP_PREFIX["neutral"]))
    # decide next question based on phase
    ph = st.session_state.phase
    if ph=="opener":
        q = random.choice(ASK_1.get(emo, ASK_1["neutral"]))
        st.session_state.phase = "explore1"
    elif ph=="explore1":
        q = random.choice(ASK_2.get(emo, ASK_2["neutral"]))
        st.session_state.phase = "explore2"
    elif ph=="explore2":
        q = random.choice(OFFER)
        st.session_state.phase = "offer"
    elif ph=="offer":
        q = random.choice(OFFER)
    else:
        q = random.choice(ASK_1["neutral"])
    return f"{prefix}\n\n{q}"

def coach_say(text:str):
    st.session_state.msgs.append({"role":"coach","text":text})

def user_say(text:str):
    st.session_state.msgs.append({"role":"user","text":text})
    st.session_state.entries.append({
        "ts": datetime.utcnow().isoformat(timespec="seconds")+"Z",
        "text": text,
        "emotion": detect_emotion(text)
    })

# ---------------- Layout ----------------
left, right = st.columns([0.60, 0.40], gap="large")

with left:
    st.subheader("Conversation")

    # render history
    for m in st.session_state.msgs:
        if m["role"]=="coach":
            st.chat_message("assistant").markdown(m["text"])
        else:
            st.chat_message("user").markdown(m["text"])

    # voice upload (optional)
    audio = st.file_uploader("Drag a voice note (mp3/wav/m4a) or type below", type=["mp3","wav","m4a"])
    transcript = None
    if audio:
        tmp = tempfile.NamedTemporaryFile(delete=False)
        tmp.write(audio.read()); tmp.flush()
        try:
            from pydub import AudioSegment; import speech_recognition as sr
            wav = tmp.name + ".wav"
            AudioSegment.from_file(tmp.name).export(wav, format="wav")
            r = sr.Recognizer()
            with sr.AudioFile(wav) as src: data = r.record(src)
            transcript = r.recognize_google(data)
            st.caption("Transcribed. You can send it or edit your own message below.")
            if st.button("Send transcription"):
                user_say(transcript)
                coach_say(empathetic_reply(transcript))
                st.rerun()
        except Exception as e:
            st.error(f"Transcription failed: {e}")

    # text input
    text = st.chat_input(placeholder="Tell me what’s going on")
    if text:
        user_say(text)
        # if we’re in "offer" phase and user chooses a path
        lower = text.lower()
        if "calm" in lower or "ground" in lower:
            st.session_state.phase = "skill"
            choice, tip = random.choice(list(CALM_TRICKS.items()))
            coach_say(f"Let’s do **{choice}**.\n\n{tip}\n\nWant another trick or keep talking?")
        elif "tiny" in lower or "action" in lower or "step" in lower:
            st.session_state.phase = "skill"
            choice, tip = random.choice(list(TINY_STEPS.items()))
            coach_say(f"Let’s try a **{choice}**.\n\n{tip}\n\nTell me when it’s done or if it feels stuck.")
        else:
            coach_say(empathetic_reply(text))
        st.rerun()

    # quick action buttons (only after some context)
    if len(st.session_state.msgs) > 2:
        c1, c2, c3 = st.columns(3)
        if c1.button("✨ Calm-down trick"):
            st.session_state.phase = "skill"
            name, tip = random.choice(list(CALM_TRICKS.items()))
            coach_say(f"Okay, **{name}**:\n\n{tip}\n\nHow do you feel after that—any shift?")
            st.rerun()
        if c2.button("🪶 Keep unpacking"):
            coach_say(random.choice(ASK_2.get(st.session_state.last_emotion, ASK_2["neutral"])))
            st.rerun()
        if c3.button("🧭 Tiny next step"):
            st.session_state.phase = "skill"
            name, tip = random.choice(list(TINY_STEPS.items()))
            coach_say(f"**Tiny step** — {name}:\n\n{tip}\n\nPing me in 5 minutes with one sentence on how it went.")
            st.rerun()

with right:
    st.subheader("Weekly glance")
    if st.session_state.entries:
        xs=[e["ts"] for e in st.session_state.entries]
        ys=[e["emotion"] for e in st.session_state.entries]
        fig = px.scatter(x=xs, y=ys, title="Emotion timeline"); st.plotly_chart(fig, use_container_width=True)
        with st.expander("Session controls"):
            buf = io.BytesIO(str(st.session_state.entries).encode())
            st.download_button("Export (.json)", buf, "soul-ink_session.json", mime="application/json")
            if st.button("Delete all data"): st.session_state.entries=[]; st.session_state.msgs=[{"role":"coach","text":"Cleared. What do you want to focus on now?"}]; st.session_state.phase="opener"; st.success("Cleared.")
    else:
        st.info("Your conversation shows up here as you chat.")
"""

open("app.py","w").write(APP)

# Kill any old servers and launch
import subprocess, time, os, signal, re
def kill(name):
    try:
        out=subprocess.check_output(["ps","-ef"]).decode()
        for ln in out.splitlines():
            if name in ln and "grep" not in ln:
                os.kill(int(re.split(r"\s+", ln)[1]), signal.SIGKILL)
    except: pass
kill("streamlit")

PORT=8501
cmd=["streamlit","run","app.py",
     "--server.port",str(PORT),
     "--server.address","0.0.0.0",
     "--server.headless","true",
     "--server.enableCORS","false",
     "--server.enableXsrfProtection","false",
     "--server.enableWebsocketCompression","false",
     "--browser.gatherUsageStats","false"]
subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
time.sleep(6)
from google.colab.output import eval_js
print("Open:", eval_js(f"google.colab.kernel.proxyPort({PORT})"))
